In [1]:
# add repo path to the system path
from pathlib import Path
import os, sys
repo_path= Path.cwd().resolve()
while '.gitignore' not in os.listdir(repo_path): # while not in the root of the repo
    repo_path = repo_path.parent #go up one level
sys.path.insert(0,str(repo_path)) if str(repo_path) not in sys.path else None

os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [4]:
import numpy as np
import cv2 as cv
import pandas as pd
from matplotlib import pyplot as plt

from detection.inference.detector import lesion_detector, post_process_pred

# Predict for test set

In [10]:
#### inputs (editable)
split_name = 'split_1'
metric_name = 'AUFROC'
####

# model inputs
config_file = repo_path / 'detection/training/config_files/fine_tuning_CEM.yaml'
min_score = 0.1 # minimum score threshold to keep the prediction

# validation data inputs
im_dir = repo_path / 'data/CDD-CESM/images/substracted' # images directory (can contain other not only test)
metadata_path = repo_path / 'data/CDD-CESM/metadata/bboxes/split_1/test_set.csv' # test data only

# get best models of the split
best_models_csv_path = repo_path / 'detection/evaluation/data/validation' / f'{split_name}/best_models/best_models_{metric_name}.csv'
best_models_csv = pd.read_csv(best_models_csv_path)
split_dir = repo_path / 'detection/training/results/split_1'

# saving directory
testing_split_dir = repo_path / 'detection/evaluation/data/testing' / f'{split_name}'
testing_split_dir.mkdir(parents=True, exist_ok=True)

for i, row in best_models_csv.iterrows():
    model_path = split_dir / row['model_type'] / f'{row["model_name"]}.pth'
    detector = lesion_detector(config_file, model_path, metadata_path, im_dir, min_score)
    detector.start_metrics()
    for im_name in detector.test_df['image_name'].unique()[0:]:
        detector.c_im_name = im_name
        detector.prepare_im_gt()
        detector.predict()
        # detector.show_c_predictions()
        # metrics computing
        used_preds = detector.compute_TP_FN_counts(show=False)
        detector.compute_FP_counts(used_preds)
    froc_info  = detector.compute_FROC()
    froc_info.to_csv(testing_split_dir / f'{row["model_type"]}.csv', index=False)

[03/06 23:26:51 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /home/ricardo/projects/CEM-Detect/detection/training/results/split_1/synthetic_improved/model_0029999.pth ...


/home/ricardo/miniconda3/envs/detectron-new/lib/python3.10/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[03/06 23:27:00 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /home/ricardo/projects/CEM-Detect/detection/training/results/split_1/real-synth_improved/model_0019999.pth ...
[03/06 23:27:07 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /home/ricardo/projects/CEM-Detect/detection/training/results/split_1/normal_fine-tuning/model_0024999.pth ...
